In [1]:
!pip install sentence-transformers faiss-cpu torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.2 MB/s eta 0:00:00


In [2]:
!pip install langchain transformers  chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.0/603.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 14.0 MB/s eta 0:00:

In [3]:
!pip install  langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00


In [4]:
import json
with open('corpus.json') as f:
    articles = json.load(f)

with open('train.json', 'r') as file:
    x = json.load(file)

In [5]:
!pip install -U sentence-transformers



In [6]:
data = {}

for i in x :
    if i['question_type'] != 'null_query':
        data[i['query']] = i['question_type']

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import re


In [8]:
label_map = {"inference_query": 0, "comparison_query": 1, "temporal_query": 2}

In [9]:
# Prepare questions and labels
questions = list(data.keys())
labels = []

labels = [label_map[label] for label in data.values()]


# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(questions, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [10]:
def simple_tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

In [11]:
# Vocabulary creation
vocab = set()
for question in questions:
    vocab.update(simple_tokenizer(question))

# Map words to indices
vocab = {word: idx + 1 for idx, word in enumerate(vocab)}  # Start indexing from 1
vocab_size = len(vocab) + 1  # Including 0 for padding

In [12]:
# Vocabulary creation
vocab = set()
for question in questions:
    vocab.update(simple_tokenizer(question))

# Map words to indices
vocab = {word: idx + 1 for idx, word in enumerate(vocab)}  # Start indexing from 1
vocab_size = len(vocab) + 1  # Including 0 for padding

In [13]:
# Tokenize and encode questions
def encode_question(question, vocab, max_len=10):
    tokens = simple_tokenizer(question)
    token_ids = [vocab.get(token, 0) for token in tokens]  # Convert tokens to ids
    token_ids = token_ids[:max_len] + [0] * (max_len - len(token_ids))  # Pad or truncate
    return token_ids

In [14]:
# Encode all questions
max_len = 10  # Max length for padding
encoded_train = [encode_question(q, vocab, max_len) for q in X_train]
encoded_val = [encode_question(q, vocab, max_len) for q in X_val]
encoded_test = [encode_question(q, vocab, max_len) for q in X_test]

# Convert to tensors
input_ids_train = torch.tensor(encoded_train, dtype=torch.long)
input_ids_val = torch.tensor(encoded_val, dtype=torch.long)
input_ids_test = torch.tensor(encoded_test, dtype=torch.long)

labels_tensor_train = torch.tensor(y_train, dtype=torch.long)
labels_tensor_val = torch.tensor(y_val, dtype=torch.long)
labels_tensor_test = torch.tensor(y_test, dtype=torch.long)

# Create datasets and DataLoaders
train_dataset = TensorDataset(input_ids_train, labels_tensor_train)
val_dataset = TensorDataset(input_ids_val, labels_tensor_val)
test_dataset = TensorDataset(input_ids_test, labels_tensor_test)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [15]:

# TextCNN model definition
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, kernel_sizes, num_filters):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embed_dim)) for k in kernel_sizes
        ])
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)
        self.dropout = nn.Dropout(0.5)

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)  # Apply convolution and ReLU
        x = F.max_pool1d(x, x.size(2)).squeeze(2)  # Max pooling
        return x

    def forward(self, x):
        x = self.embedding(x)  # (batch_size, max_len, embed_dim)
        x = x.unsqueeze(1)  # Add a channel dimension for Conv2D (batch_size, 1, max_len, embed_dim)
        x = torch.cat([self.conv_and_pool(x, conv) for conv in self.convs], 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


In [16]:
# Initialize the TextCNN model
embed_dim = 50  # Embedding size
num_classes = 3  # Number of output classes (inference, comparison, temporal)
kernel_sizes = [2, 3, 4]  # Different kernel sizes for convolution
num_filters = 100  # Number of filters per kernel size

model1 = TextCNN(vocab_size, embed_dim, num_classes, kernel_sizes, num_filters)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

In [17]:

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model1.train()
    total_loss = 0
    for batch in train_loader:
        input_ids_batch, labels_batch = batch

        # Forward pass
        outputs = model1(input_ids_batch)
        loss = criterion(outputs, labels_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')

Epoch [1/20], Loss: 194.6205
Epoch [2/20], Loss: 51.2041
Epoch [3/20], Loss: 29.7480
Epoch [5/20], Loss: 14.0650
Epoch [6/20], Loss: 16.1002
Epoch [7/20], Loss: 19.1586
Epoch [8/20], Loss: 12.9028
Epoch [9/20], Loss: 27.5504
Epoch [10/20], Loss: 12.6616
Epoch [11/20], Loss: 19.6253
Epoch [12/20], Loss: 12.8117
Epoch [13/20], Loss: 26.8751
Epoch [14/20], Loss: 3.7007
Epoch [15/20], Loss: 15.7614
Epoch [16/20], Loss: 17.3067
Epoch [17/20], Loss: 13.6011
Epoch [18/20], Loss: 8.8928
Epoch [19/20], Loss: 9.5169
Epoch [20/20], Loss: 2.8663


In [18]:
model1.eval()
val_loss = 0
with torch.no_grad():
    for batch in val_loader:
        input_ids_batch, labels_batch = batch

        outputs = model1(input_ids_batch)
        loss = criterion(outputs, labels_batch)
        val_loss += loss.item()

    print(f'Validation Loss: {val_loss:.4f}')


Validation Loss: 27.2099


In [19]:
#  Function to preprocess a question (tokenize and encode)

def preprocess_question(question, vocab, max_len=10):
    def simple_tokenizer(text):
        # Tokenize text into words
        return re.findall(r'\b\w+\b', text.lower())

    # Tokenize and encode question into token IDs using the vocabulary
    tokens = simple_tokenizer(question)
    token_ids = [vocab.get(token, 0) for token in tokens]  # Convert tokens to ids (0 if not found)
    token_ids = token_ids[:max_len] + [0] * (max_len - len(token_ids))  # Pad or truncate to max_len
    return torch.tensor([token_ids], dtype=torch.long)  # Return as tensor with batch size of 1

# Function to classify the question type
def classify_question(model, question, vocab, label_map):
    model1.eval()  # Set model to evaluation mode

    # Preprocess the input question
    input_ids = preprocess_question(question, vocab)
    # input_ids =encode_question(question,vocab)

    # Forward pass: Pass the question through the model
    with torch.no_grad():  # Disable gradient computation
        outputs = model1(input_ids)  # Get model predictions
        predicted_label_idx = torch.argmax(outputs, dim=1).item()  # Get the predicted class index

    # Convert the predicted index back to the label
    inverse_label_map = {v: k for k, v in label_map.items()}  # Reverse the label map
    predicted_label = inverse_label_map[predicted_label_idx]  # Get label from index

    return predicted_label

# Example usage
question = "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"  # Example question

# Call the classify_question function to get the type
predicted_type = classify_question(model1, question, vocab, label_map)
print(f"The predicted type of the question is: {predicted_type}")

The predicted type of the question is: inference_query


In [20]:
from sentence_transformers import SentenceTransformer


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [21]:
from sentence_transformers import SentenceTransformer
import os

# os.environ['CUDA_LAUNCH_BLOCKING']="1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"


model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2',device=0)
chunks_with_metadata = []
chunks = []



def chunk_text(text, max_sentences=2):
    sentences = re.split(r'(?<=[.!?])\s+', text)  # Split by full stop, question mark, or exclamation mark
    chunks = []
    current_chunk = []
    sentence_count = 0

    for sentence in sentences:
        if sentence_count == max_sentences:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            sentence_count = 1  # Reset count for the new chunk
        else:
            current_chunk.append(sentence)
            sentence_count += 1

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

for article in articles:
    body = article['body']
    article_chunks = chunk_text(body)

    for chunk in article_chunks:
        chunks.append(chunk)
        chunks_with_metadata.append({
            "chunk": chunk,
            "title": article['title'],
            "author": article['author'],
            "source": article['source'],
            "published_at": article['published_at'],
            "category": article['category'],
            "url": article['url']
        })

# Embed the chunks in batches
batch_size = 32
chunk_embeddings = []

for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i+batch_size]
    try:
        batch_embeddings = model.encode(batch, convert_to_tensor=True)
        chunk_embeddings.extend(batch_embeddings)
    except Exception as e:
        print(f"Error encoding batch {i}-{i+batch_size}: {str(e)}")

# Convert list of tensors to a single tensor
chunk_embeddings = torch.stack(chunk_embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
import torch
from transformers import pipeline
from sentence_transformers import util

# Force to use CPU by setting device=-1
device = 0

# Initialize the summarization pipeline with CPU support
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
# Function to retrieve relevant chunks based on user input (search term)
def retrieve_relevant_chunks(query, top_k=4):
    # Embed the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute cosine similarities
    cos_scores = util.pytorch_cos_sim(query_embedding, chunk_embeddings)[0]
    # print(max(cos_scores))
    max_score, max_idx = torch.max(cos_scores, dim=0)
    print("cos_scores: ",max_score)

    if max_score < 0.5:
      return_list = []
      return return_list


    # Get the top K results
    top_results = torch.topk(cos_scores, k=top_k)

    # Gather relevant chunks with their metadata
    relevant_chunks = {}
    for score, idx in zip(top_results[0], top_results[1]):
        doc_id = chunks_with_metadata[idx]["url"]  # Use URL or any unique identifier for the document
        chunk_data = {
            "chunk": chunks[idx],
            "title": chunks_with_metadata[idx]["title"],
            "author": chunks_with_metadata[idx]["author"],
            "url": chunks_with_metadata[idx]["url"],  # Add URL
            "source": chunks_with_metadata[idx]["source"],
            "category": chunks_with_metadata[idx]["category"],
            "published_at": chunks_with_metadata[idx]["published_at"],
            "score": score.item()  # Convert score to a float
        }
        relevant_chunks[doc_id] = chunk_data

    return list(relevant_chunks.values())  # Convert back to a list for consistency



def summarize_chunks(relevant_chunks, max_length=130, min_length=30):
    # Combine all relevant chunks into a single text
    combined_text = " ".join([chunk['chunk'] for chunk in relevant_chunks])

    # Perform summarization
    summary = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']

    return summary



config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [23]:
from transformers import pipeline
import torch

In [24]:
!pip install flask-ngrok
!pip install pyngrok

In [25]:
from pyngrok import ngrok
ngrok.set_auth_token("2nM4qZgRikmYOcap5gcC38zT57i_3cHVziYb12tcgrvm5Sgrt")

In [26]:
!pip install CORS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.3/254.3 kB 21.0 MB/s eta 0:00:00


In [27]:
!pip install flask-cors


In [28]:
from flask_cors import CORS


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from transformers import pipeline
import torch

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Check if GPU is available
device = 0 if torch.cuda.is_available() else -1

# Initialize the pipelines
question_answerer = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad", device=0)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)

def handle_temporal_query(question):
    candidate_labels = ['Yes', 'No']
    response = classifier(question, candidate_labels)
    return response['labels'][0]  # Highest scoring label (Yes/No)

def handle_comparison_query(question, context=""):
    prompt = f"Answer the question with yes or no:\n\nQuestion: {question}\nAnswer:"
    response = question_answerer({
        'question': prompt,
        'context': context or " "
    })
    answer = response['answer'].strip().lower()

    if 'yes' in answer:
        return 'Yes'

    elif 'no' in answer:
        return 'No'

    else:
        return 'No'


# Health check endpoint
@app.route('/health-check', methods=['GET'])
def health_check():
    return jsonify({"status": "healthy"}), 200

# Endpoint to process questions
@app.route('/process-question', methods=['POST'])
def process_question():
    data = request.json
    question = data.get('question')
    query = question
    question = question

    if not question:
        return jsonify({"error": "No question provided"}), 400

    # Retrieve relevant chunks (assuming you have the retrieve_relevant_chunks function defined)
    relevant_chunks = retrieve_relevant_chunks(question)

    if len(relevant_chunks) == 0:
        output = {
            "query": question,
            "answer": "Insufficient information.",
            "question_type": "null_query",
            "evidence_list": []
        }
        return jsonify(output)

    # Generate summary from relevant chunks
    summary = summarize_chunks(relevant_chunks)
    # Perform question answering using the summary as context
    result = question_answerer(question=question, context=summary)['answer']
    question_type = classify_question(model1, question, vocab, label_map)

    if question_type == "temporal_query":
        result = handle_temporal_query(question + summary)
    elif question_type == "comparison_query":
        result = handle_comparison_query(question, context=summary)

    output = {
        "query": query,
        "answer": result,
        "question_type": question_type,
        "evidence_list": [
            {
                "title": chunk['title'],
                "author": chunk['author'],
                "url": chunk['url'],
                "source": chunk['source'],
                "category": chunk['category'],
                "published_at": chunk['published_at'],
                "fact": (chunk['chunk'])  # Wrap the fact text
            }
            for chunk in relevant_chunks
        ]
    }

    return jsonify(output)

# Set up ngrok
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"http://127.0.0.1:5000\" -> \"{public_url}\"")

# Run the Flask app
if __name__ == '__main__':
    app.run()


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

INFO:werkzeug:127.0.0.1 - - [15/Oct/2024 06:10:56] "POST /process-question HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2024 06:15:06] "OPTIONS /process-question HTTP/1.1" 200 -


cos_scores:  tensor(0.6976, device='cuda:0')


INFO:werkzeug:127.0.0.1 - - [15/Oct/2024 06:15:08] "POST /process-question HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2024 06:15:42] "OPTIONS /process-question HTTP/1.1" 200 -


cos_scores:  tensor(0.6359, device='cuda:0')


INFO:werkzeug:127.0.0.1 - - [15/Oct/2024 06:15:44] "POST /process-question HTTP/1.1" 200 -
